For this project, you will be working with the Hop Teaming dataset, a dataset which aims to capture referrals between healthcare providers based on Medicare claims. 
We will be using multiple data sets HOP_teaming(hospitals referal), NEPPS(doctors NPI and specifics), Taxonomy(details about the taxonomy code in NPESS) and the ZIP_CBSA(locations and connecting to doctors)

In [59]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [60]:
#reading HOP.csv for overveiw and glance
chunks = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv',
           chunksize = 10000)

In [61]:
chunks

In [62]:
chunk = next(chunks)

In [63]:
chunk

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880
...,...,...,...,...,...,...
9995,1508026980,1730477589,36,46,64.239,77.845
9996,1497997050,1730477589,38,43,40.395,60.289
9997,1497977268,1730477761,14,14,37.857,72.533
9998,1508024217,1730477811,12,12,59.000,46.043


We want to eliminate "accidental" referrals, so filter the hop teaming data so that the transaction_count is at least 50 and the average_day_wait is less than 50.

In [64]:
chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait']<50)]

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
7,1508085911,1730166125,58,67,23.925,43.923
10,1508167040,1730166125,51,51,28.196,52.876
15,1508863549,1730166125,340,391,18.302,42.422
18,1508867870,1730166125,50,79,12.658,26.402
25,1508011040,1730166224,132,145,8.579,28.053
...,...,...,...,...,...,...
9951,1508804113,1730476490,97,102,32.882,54.294
9976,1508834862,1730476979,53,60,34.983,54.899
9979,1508118555,1730476995,84,91,22.791,38.622
9982,1508131087,1730477225,96,323,3.613,21.439


In [65]:
#Reading the NEPSS_NPI for glance
chunks_npi = pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
           chunksize = 10000)

In [66]:
chunk_npi = next(chunks_npi) #to explore the chunk as df
chunk_npi

C:\Users\upadh\AppData\Local\Temp\ipykernel_384\3734988901.py:1: DtypeWarning: Columns (17,24,64,68,71,72,73,74,75,76,77,78,79,80,81,82,83,85,86,87,89,90,147,151,155,157,159,161,163,165,167,169,170,171,174,175,178,314) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk_npi = next(chunks_npi)


,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date
0,1740284231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1346245800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1487650776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1033113022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1043216138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1558360792,1.0,NaN,NaN,NaN,RODGERS,SUSAN,V,MRS.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,1467451609,1.0,NaN,NaN,NaN,HILTON,BRET,D,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,1346249596,1.0,NaN,NaN,NaN,STANLEY,GARY,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,1982603130,1.0,NaN,NaN,NaN,SLETTEN,CHRISTINA,A.,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
#getting the feel of the read in chunk 
chunk_npi.info()
chunk_npi.shape #it has 10K rows and 330 columns
list(chunk_npi.columns) getting the name of the columns as a list to see

(10000, 330)

In [126]:
#getting the feel of the read in chunk 
chunk_npi['Provider Organization Name (Legal Business Name)'].value_counts()
chunk_npi['Healthcare Provider Taxonomy Code_1'].value_counts()
chunk_npi[ 'Healthcare Provider Primary Taxonomy Switch_1'].value_counts()

Y    9001
X     374
N      70
Name: Healthcare Provider Primary Taxonomy Switch_1, dtype: int64

In [68]:
#subsetting the df to just keeo the coulmns needed.
npi = chunk_npi[['NPI',
 'Entity Type Code',
 'Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider First Line Business Practice Location Address',
 'Provider Second Line Business Practice Location Address',
 'Provider Business Practice Location Address City Name',
 'Provider Business Practice Location Address State Name',
 'Provider Business Practice Location Address Postal Code',
           'Healthcare Provider Taxonomy Code_1',
           'Healthcare Provider Primary Taxonomy Switch_1',
           'Healthcare Provider Taxonomy Code_2',
           'Healthcare Provider Primary Taxonomy Switch_2', 
           'Healthcare Provider Taxonomy Code_3',
           'Healthcare Provider Primary Taxonomy Switch_3',     
           'Healthcare Provider Taxonomy Code_4',
           'Healthcare Provider Primary Taxonomy Switch_4',
           'Healthcare Provider Taxonomy Code_5',
           'Healthcare Provider Primary Taxonomy Switch_5',
            'Healthcare Provider Taxonomy Code_6',
           'Healthcare Provider Primary Taxonomy Switch_6',
        'Healthcare Provider Taxonomy Code_7',
           'Healthcare Provider Primary Taxonomy Switch_7',
                 'Healthcare Provider Taxonomy Code_8',
           'Healthcare Provider Primary Taxonomy Switch_8',
                 'Healthcare Provider Taxonomy Code_9',
           'Healthcare Provider Primary Taxonomy Switch_9',
                 'Healthcare Provider Taxonomy Code_10',
           'Healthcare Provider Primary Taxonomy Switch_10',
                 'Healthcare Provider Taxonomy Code_11',
           'Healthcare Provider Primary Taxonomy Switch_11',
                 'Healthcare Provider Taxonomy Code_12',
           'Healthcare Provider Primary Taxonomy Switch_12',
                 'Healthcare Provider Taxonomy Code_13',
           'Healthcare Provider Primary Taxonomy Switch_13',
                 'Healthcare Provider Taxonomy Code_14',
           'Healthcare Provider Primary Taxonomy Switch_14',
                 'Healthcare Provider Taxonomy Code_15',
           'Healthcare Provider Primary Taxonomy Switch_15'
            ]]

In [70]:
#scratch work to iterate over the coulmns and print the column name and values
for column in (npi.columns):
    
    # Select column contents by column
    # name using [] operator
    columnSeriesObj = npi[column]
    print('Column Name : ', column)
    print('Column Contents : ', columnSeriesObj.values)

Column Name :  NPI
Column Contents :  [1740284231 1346245800 1487650776 ... 1346249596 1982603130 1336148584]
Column Name :  Entity Type Code
Column Contents :  [nan nan nan ...  1.  1.  1.]
Column Name :  Provider Organization Name (Legal Business Name)
Column Contents :  [nan nan nan ... nan nan nan]
Column Name :  Provider Last Name (Legal Name)
Column Contents :  [nan nan nan ... 'STANLEY' 'SLETTEN' 'HANCOCK']
Column Name :  Provider First Name
Column Contents :  [nan nan nan ... 'GARY' 'CHRISTINA' 'WILLIAM']
Column Name :  Provider Middle Name
Column Contents :  [nan nan nan ... 'E' 'A.' 'E']
Column Name :  Provider Name Prefix Text
Column Contents :  [nan nan nan ... nan 'DR.' 'DR.']
Column Name :  Provider Name Suffix Text
Column Contents :  [nan nan nan ... nan nan nan]
Column Name :  Provider Credential Text
Column Contents :  [nan nan nan ... 'M.D.' 'DDS' 'MD']
Column Name :  Provider First Line Business Practice Location Address
Column Contents :  [nan nan nan ... '920 OLIVE

In [129]:
x = []

for column in (npi.columns):
#for i in range (1,16):
    columnSeriesObj = npi[column]
    if f'Healthcare Provider Primary Taxonomy Switch_1' == 'Y',
        x.append['Healthcare Provider Taxonomy Code_1'.values]
    elif
        f'Healthcare Provider Primary Taxonomy Switch_2' == 'Y',
        x.append['Healthcare Provider Taxonomy Code_2'.values]
    elif
    f'Healthcare Provider Primary Taxonomy Switch_3' == 'Y',
    x.append['Healthcare Provider Taxonomy Code_3'.values]
    elif
    f'Healthcare Provider Primary Taxonomy Switch_4' == 'Y',
    x.append['Healthcare Provider Taxonomy Code_4'.values]
    elif
    f'Healthcare Provider Primary Taxonomy Switch_3' == 'Y',
    x.append['Healthcare Provider Taxonomy Code_3'.values]
    elif
    f'Healthcare Provider Primary Taxonomy Switch_3' == 'Y',
    x.append['Healthcare Provider Taxonomy Code_3'.values]
    elif
    f'Healthcare Provider Primary Taxonomy Switch_3' == 'Y',
    x.append['Healthcare Provider Taxonomy Code_3'.values]
    elif
    f'Healthcare Provider Primary Taxonomy Switch_3' == 'Y',
    x.append['Healthcare Provider Taxonomy Code_3'.values]
    else
    

print(x)
#use the loop for selecting the rows with switch as 'Y' and collect all codes in code column to create tiny tables and then concat them

SyntaxError: invalid syntax (4129370470.py, line 6)

In [136]:
#Steps for my loop
x = []
for i in range [1,16]:
    if f'Healthcare Provider Primary Taxonomy Switch_{i}' == 'Y':
        x.append[f'Healthcare Provider Taxonomy Code_{i}'.values]
    elif f'Healthcare Provider Primary Taxonomy Switch_{i+1}' == 'Y':
        x.append[f'Healthcare Provider Taxonomy Code_{i+1}'.values]
    else:
        x.append['none']

TypeError: 'type' object is not subscriptable

In [137]:
# to get all the "Y" taxonomy codes
x = []
for i in range (1,16):
    x.append(npi[npi[f'Healthcare Provider Primary Taxonomy Switch_{i}' ]== 'Y'][['NPI',
         'Entity Type Code',
         'Provider Organization Name (Legal Business Name)',
         'Provider Last Name (Legal Name)',
         'Provider First Name',
         'Provider Middle Name',
         'Provider Name Prefix Text',
         'Provider Name Suffix Text',
         'Provider Credential Text',
         'Provider First Line Business Practice Location Address',
         'Provider Second Line Business Practice Location Address',
         'Provider Business Practice Location Address City Name',
         'Provider Business Practice Location Address State Name',
         'Provider Business Practice Location Address Postal Code', 
        f'Healthcare Provider Taxonomy Code_{i}']].rename(columns={f'Healthcare Provider Taxonomy Code_{i}':'Taxonomy_code'}))

In [117]:
#x is coming as multiple df so to stich them together use the pd.concat 
yes_code = pd.concat(x)
yes_code.shape

(9066, 15)

In [120]:
pd.merge(left = npi[['NPI',
         'Entity Type Code',
         'Provider Organization Name (Legal Business Name)',
         'Provider Last Name (Legal Name)',
         'Provider First Name',
         'Provider Middle Name',
         'Provider Name Prefix Text',
         'Provider Name Suffix Text',
         'Provider Credential Text',
         'Provider First Line Business Practice Location Address',
         'Provider Second Line Business Practice Location Address',
         'Provider Business Practice Location Address City Name',
         'Provider Business Practice Location Address State Name',
         'Provider Business Practice Location Address Postal Code']], right = yes_code, how = "left").drop_duplicates()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Taxonomy_code
0,1740284231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1346245800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1487650776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1033113022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1043216138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1558360792,1.0,NaN,RODGERS,SUSAN,V,MRS.,NaN,CPHT,1919 S LOOP 256,NaN,PALESTINE,TX,758015915.0,183700000X
9996,1467451609,1.0,NaN,HILTON,BRET,D,NaN,NaN,PA,280 RIVER PARK DR,SUITE 120,PROVO,UT,846045764.0,363A00000X
9997,1346249596,1.0,NaN,STANLEY,GARY,E,NaN,NaN,M.D.,920 OLIVER RD,SUITE 3,MONROE,LA,712015702.0,208000000X
9998,1982603130,1.0,NaN,SLETTEN,CHRISTINA,A.,DR.,NaN,DDS,6685 DELMONICO DR,SUITE C,COLORADO SPRINGS,CO,809196802.0,122300000X


In [122]:
npi[npi['NPI']== 1477551547]
npi.loc[8422]
npi['NPI'].nunique()

10000

In [93]:
npi['Healthcare Provider Primary Taxonomy Switch_1'].value_counts(dropna = False)

Y      9001
NaN     555
X       374
N        70
Name: Healthcare Provider Primary Taxonomy Switch_1, dtype: int64

In [106]:
#finding the switch with X
with_x = []
for i in range (1,16):
    with_x.append(npi[npi[f'Healthcare Provider Primary Taxonomy Switch_{i}' ]== 'N'][['NPI',
         'Entity Type Code',
         'Provider Organization Name (Legal Business Name)',
         'Provider Last Name (Legal Name)',
         'Provider First Name',
         'Provider Middle Name',
         'Provider Name Prefix Text',
         'Provider Name Suffix Text',
         'Provider Credential Text',
         'Provider First Line Business Practice Location Address',
         'Provider Second Line Business Practice Location Address',
         'Provider Business Practice Location Address City Name',
         'Provider Business Practice Location Address State Name',
         'Provider Business Practice Location Address Postal Code', 
        f'Healthcare Provider Taxonomy Code_{i}']].rename(columns={f'Healthcare Provider Taxonomy Code_{i}':'Taxonomy_code'}))

In [107]:
pd.concat(with_x)

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Taxonomy_code
597,1083617542,1.0,NaN,PARKER,MELODIE,NaN,NaN,NaN,LCSW,48 BURD ST,STE 201,NYACK,NY,109603226.0,1041C0700X
740,1336142231,1.0,NaN,RODRIGUEZ,MATTHEW,JOHN,DR.,NaN,DOCTOR OF PHARMACY,1700 CERRILLOS RD,NaN,SANTA FE,NM,875053554.0,183500000X
832,1013910892,1.0,NaN,LEJEUNE,EMIKO,JANE,NaN,NaN,M.S.,1740 MEMORIAL DR,NaN,CLARKSVILLE,TN,370434561.0,231H00000X
988,1003819632,1.0,NaN,ROYER,GAYLA,M,DR.,NaN,MD,1317 4TH AVE S,NaN,BIRMINGHAM,AL,352331408.0,208C00000X
1151,1477556652,1.0,NaN,BRUNO,EDWIN,JAY,DR.,NaN,D.P.M.,277 STATE ST,STE 1B,BANGOR,ME,44015440.0,213E00000X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,1538162540,1.0,NaN,ROUCHON,DEBORAH,S,MS.,NaN,CRNA,1009 OAK CIR,NaN,PALM HARBOR,FL,346836627.0,367500000X
1063,1538162540,1.0,NaN,ROUCHON,DEBORAH,S,MS.,NaN,CRNA,1009 OAK CIR,NaN,PALM HARBOR,FL,346836627.0,367500000X
1063,1538162540,1.0,NaN,ROUCHON,DEBORAH,S,MS.,NaN,CRNA,1009 OAK CIR,NaN,PALM HARBOR,FL,346836627.0,367500000X
1063,1538162540,1.0,NaN,ROUCHON,DEBORAH,S,MS.,NaN,CRNA,1009 OAK CIR,NaN,PALM HARBOR,FL,346836627.0,367500000X


In [ ]:
with_x = []
for i in range (1,16):
    with_x.append(npi[npi[f'Healthcare Provider Primary Taxonomy Switch_{i}' ] != 'N'][['NPI',
         'Entity Type Code',
         'Provider Organization Name (Legal Business Name)',
         'Provider Last Name (Legal Name)',
         'Provider First Name',
         'Provider Middle Name',
         'Provider Name Prefix Text',
         'Provider Name Suffix Text',
         'Provider Credential Text',
         'Provider First Line Business Practice Location Address',
         'Provider Second Line Business Practice Location Address',
         'Provider Business Practice Location Address City Name',
         'Provider Business Practice Location Address State Name',
         'Provider Business Practice Location Address Postal Code', 
        f'Healthcare Provider Taxonomy Code_{i}']].rename(columns={f'Healthcare Provider Taxonomy Code_{i}':'Taxonomy_code'}))

In [138]:
pd.merge(left = npi[['NPI',
         'Entity Type Code',
         'Provider Organization Name (Legal Business Name)',
         'Provider Last Name (Legal Name)',
         'Provider First Name',
         'Provider Middle Name',
         'Provider Name Prefix Text',
         'Provider Name Suffix Text',
         'Provider Credential Text',
         'Provider First Line Business Practice Location Address',
         'Provider Second Line Business Practice Location Address',
         'Provider Business Practice Location Address City Name',
         'Provider Business Practice Location Address State Name',
         'Provider Business Practice Location Address Postal Code']], right = pd.concat(x), how = "left").drop_duplicates()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Taxonomy_code
0,1740284231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1346245800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1487650776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1033113022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1043216138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1558360792,1.0,NaN,RODGERS,SUSAN,V,MRS.,NaN,CPHT,1919 S LOOP 256,NaN,PALESTINE,TX,758015915.0,183700000X
9996,1467451609,1.0,NaN,HILTON,BRET,D,NaN,NaN,PA,280 RIVER PARK DR,SUITE 120,PROVO,UT,846045764.0,363A00000X
9997,1346249596,1.0,NaN,STANLEY,GARY,E,NaN,NaN,M.D.,920 OLIVER RD,SUITE 3,MONROE,LA,712015702.0,208000000X
9998,1982603130,1.0,NaN,SLETTEN,CHRISTINA,A.,DR.,NaN,DDS,6685 DELMONICO DR,SUITE C,COLORADO SPRINGS,CO,809196802.0,122300000X
